In [2]:
def get_fg(image_path:str,fg_x:int,fg_y:int,fg_width:int,fg_height:int, accuracy:int):
    import cv2 as cv
    import numpy as np
#    print(f"your OpenCV version is {cv.__version__}")
    image = cv.imread(image_path)
    if image is not None:
        i_mask = np.zeros(image.shape[:2],np.uint8)
        i_rect = (fg_x,fg_y,fg_width,fg_height)
        i_bg_model = np.zeros((1,65),np.float64)
        i_fg_model = np.zeros((1,65),np.float64)
        L_ITERATIONS = accuracy # = epochs in ANN
        i_modes = [cv.GC_INIT_WITH_RECT, cv.GC_INIT_WITH_MASK]
        cv.grabCut(
            img = image,
            mask = i_mask,
            rect = i_rect,
            bgdModel = i_bg_model,
            fgdModel = i_fg_model,
            iterCount = L_ITERATIONS,
            mode = i_modes[0])
        i_flagged_mask = np.where((i_mask==2)|(i_mask==0),0,1).astype('uint8')
        return image * i_flagged_mask[:,:,np.newaxis]
    else:
        raise ValueError("Image not found")

In [3]:
def show_image(image, title:str=None):
    import cv2 as cv
    import matplotlib.pyplot as plt
    %matplotlib inline
    plt.imshow(cv.cvtColor(image, cv.COLOR_BGR2RGB))
    if title is not None:
        plt.title(title)
    plt.show()

In [5]:
def find_faces(image_path:str, scale_factor: float=1.1, min_neighbours: int=4):
    import cv2 as cv
    
    if (image := cv.imread(image_path)) is not None:
        cascade = cv.CascadeClassifier(cv.data.haarcascades + "haarcascade_frontalface_default.xml")
        result = image.copy()
        detected = cascade.detectMultiScale(
            image = cv.cvtColor(
                src = image, 
                code = cv.COLOR_BGR2GRAY
            ), 
            scaleFactor = scale_factor, 
            minNeighbors = min_neighbours
        )
        for (x, y, w, h) in detected:
            cv.rectangle(
                img = result, 
                pt1 = (x, y), 
                pt2 = (x+w, y+h), 
                color = (0, 255, 0), 
                thickness = 2)
            
        return result, len(detected)
    else:
        raise ValueError(f"Image not found: {image_path}")

In [6]:
def show_with_count(detection_result: tuple):
    show_image(detection_result[0], f"Faces detected: {detection_result[1]}")

In [7]:
def video_emotion_overlay(input_src, output_src):
    import cv2
    import numpy as np
    import deepface as DeepFace
    
    cap = cv2.VideoCapture(input_src)
    cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
    
    output_path = output_src
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    writer = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), 20, (width, height))
    
    # Does whole video. Takes some time and memory
    r, frame_source = cap.read()
    while r is not False:    
        frame = cv2.cvtColor(frame_source, cv2.COLOR_BGR2GRAY)
        face = cascade.detectMultiScale(frame, 1.1, 4)
        if np.any(face):
            for (x, y, w, h) in face:
                emotion = DeepFace.analyze(frame_source, actions=("emotion", ), silent=True)
                text = f"Emotion is {emotion[0]['dominant_emotion']}"
                cv2.putText(frame_source, text, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36, 255, 12), 2)
                cv2.rectangle(frame_source, (x, y), (x+w, y+h), (0, 255, 0), 2)
                writer.write(frame_source)
        r, frame_source = cap.read()
    
    cap.release()
    writer.release()